In [ ]:
from google.colab import drive
drive.mount('/content/drive')
% cd /content/drive/MyDrive/HASOC Project Folder/2021 Dataset/CodeMix Dataset/

Mounted at /content/drive
/content/drive/MyDrive/HASOC Project Folder/2021 Dataset/CodeMix Dataset


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None) 

In [ ]:
from glob import glob
import re
import json

train_directories = []
for i in glob("data/train/*/"):
    for j in glob(i+'*/'):
        train_directories.append(j)

In [ ]:
data = []
for i in train_directories:
    with open(i+'data.json', encoding='utf-8') as f:
        data.append(json.load(f))
labels = []
for i in train_directories:
    with open(i+'labels.json', encoding='utf-8') as f:
        labels.append(json.load(f))

In [ ]:
def p2_flatten(d, l):

    flat_text = []

    flat_text.append({
        "post": d['tweet'],
        "context":"",
        'text': "",
        'label': l[d['tweet_id']]
    })

    for i in d['comments']:

        flat_text.append({
            "post": d['tweet'],
            'context': i["tweet"],
            'text': "",
            'label': l[i['tweet_id']]
        })

        if 'replies' in i.keys():
            for j in i['replies']:
                previous_reply = ""
                flat_text.append({
                    "post": d['tweet'],
                    'context': i["tweet"] + previous_reply,
                    'text': j['tweet'],
                    'label': l[j['tweet_id']]
                })
                previous_reply = previous_reply + " [SEP] "+ j["tweet"]

    return flat_text

In [ ]:
data_label = []

for i in range(len(labels)):
    for j in p2_flatten(data[i], labels[i]):
        data_label.append(j)

In [ ]:
df = pd.DataFrame(data_label, columns = data_label[0].keys(), index = None)

In [ ]:
df["label"] = df["label"].map({"HOF":1,"NONE":0})

In [ ]:
df.to_pickle("train_hierarchial.pkl")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2819 entries, 0 to 2818
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   post     2819 non-null   object
 1   context  2819 non-null   object
 2   text     2819 non-null   object
 3   label    2819 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 88.2+ KB


In [ ]:
train = pd.read_pickle("train_hierarchial.pkl")

In [ ]:
train

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
post,context, label = train.post.values, train.context, train.label

In [ ]:
#check for GPU access
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 55.4 MB/s 
     |████████████████████████████████| 596 kB 35.5 MB/s 
     |████████████████████████████████| 3.3 MB 36.0 MB/s 
     |████████████████████████████████| 61 kB 482 kB/s 


In [ ]:
from transformers import BertModel
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

class CustomBERTModel(nn.Module):
    def __init__(self):
          super(CustomBERTModel, self).__init__()

          self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

          self.bert = BertModel.from_pretrained('bert-base-uncased')
         
          self.linear1 = nn.Linear(1536, 768)
          self.linear2 = nn.Linear(768, 300)
          self.linear3 = nn.Linear(1068, 768)
          self.linear4 = nn.Linear(768,2)

    def bert_representations(self, sentence):
      input = self.tokenizer(sentence, return_tensors = "pt")
      outputs =  self.bert(**input)
      return outputs.last_hidden_state[:,0,:]

    def concatenate(self, tensor1, tensor2):
      return torch.cat((tensor1, tensor2), 1)
      
    def forward(self, post, context, text, label, only_post = False, post_context=False):

      if only_post:
        post = self.bert_representations(post)
        linear4_output = self.linear4(post)

      elif post_context:
        post, context = self.bert_representations(post), self.bert_representations(context)
        post_context = self.concatenate(post, context)
        linear1_output = self.linear1(post_context)
        linear4_output = self.linear4(linear1_output)

      else:
        post, context, text = self.bert_representations(post), self.bert_representations(context), self.bert_representations(text)

        context_text = self.concatenate(context, text)

        linear1_output = self.linear1(context_text)

        linear2_output = self.linear2(linear1_output)

        post_context_text = self.concatenate(post, linear2_output)

        linear3_output = self.linear3(post_context_text)

        linear4_output = self.linear4(linear3_output)

      return linear4_output

In [ ]:
model = CustomBERTModel()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 207 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
epochs = 4

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))


print("Training....")

for epoch in range(epochs):
    total_train_loss=0

    print("==========Epochs:{}===========".format(epoch)) 

    for post, context,text, label in zip(train["post"], train["context"], train["text"], train["label"]):

      if context=="":
        logits = model.forward(post, context,text, label, only_post=True)

      elif context!="" and text=="":
        logits = model.forward(post, context,text, label,post_context=True)

      else:
        logits = model.forward(post, context,text, label)

      print(logits)
      optimizer.zero_grad() 

      one_hot = torch.zeros(1, 2)
      one_hot[0, label] = 1
        
      loss = criterion(logits, one_hot)

      total_train_loss += loss
 
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()

    print("Training Loss: {}".format((total_train_loss/len(df)).item()))
    print("=========================")

Training....
==========Epochs:0===========
tensor([[0.4296, 0.0394]], grad_fn=<AddmmBackward0>)
tensor([[-0.0083, -0.2433]], grad_fn=<AddmmBackward0>)
tensor([[ 5.9600, -5.4087]], grad_fn=<AddmmBackward0>)
tensor([[-0.1721,  0.1087]], grad_fn=<AddmmBackward0>)
tensor([[-4.7218,  4.3171]], grad_fn=<AddmmBackward0>)
tensor([[-6.9588,  6.3769]], grad_fn=<AddmmBackward0>)
tensor([[ 2.9862, -3.0744]], grad_fn=<AddmmBackward0>)
tensor([[-0.9076,  0.7013]], grad_fn=<AddmmBackward0>)
tensor([[-4.0221,  3.5611]], grad_fn=<AddmmBackward0>)
tensor([[-1.5806,  1.2522]], grad_fn=<AddmmBackward0>)
0.008275897242128849
Training....
==========Epochs:1===========
tensor([[-0.8340,  0.5336]], grad_fn=<AddmmBackward0>)
tensor([[ 1.0934, -1.0150]], grad_fn=<AddmmBackward0>)
tensor([[ 8.3489, -7.7873]], grad_fn=<AddmmBackward0>)
tensor([[ 4.8184, -4.7326]], grad_fn=<AddmmBackward0>)
tensor([[ 1.1642, -1.2842]], grad_fn=<AddmmBackward0>)
tensor([[-5.1292,  4.5427]], grad_fn=<AddmmBackward0>)
tensor([[-6.395

In [ ]:
# Testing Setup

In [ ]:
from glob import glob
import re
import json

train_directories = []
for i in glob("test/*/"):
    for j in glob(i+'*/'):
        train_directories.append(j)

In [ ]:
data = []
for i in train_directories:
    with open(i+'data.json', encoding='utf-8') as f:
        data.append(json.load(f))
labels = []
for i in train_directories:
    with open(i+'labels.json', encoding='utf-8') as f:
        labels.append(json.load(f))

In [ ]:
def p2_flatten(d, l):

    flat_text = []

    flat_text.append({
        "post": d['tweet'],
        "context":"",
        'text': "",
        'label': l[d['tweet_id']]
    })

    for i in d['comments']:

        flat_text.append({
            "post": d['tweet'],
            'context': i["tweet"],
            'text': "",
            'label': l[i['tweet_id']]
        })
        if 'replies' in i.keys():
            for j in i['replies']:
                previous_reply = ""
                flat_text.append({
                    "post": d['tweet'],
                    'context': i["tweet"] + previous_reply,
                    'text': j['tweet'],
                    'label': l[j['tweet_id']]
                })
                previous_reply = previous_reply + " [SEP] "+ j["tweet"]

    return flat_text

In [ ]:
data_label = []

for i in range(len(labels)):
    for j in p2_flatten(data[i], labels[i]):
        data_label.append(j)

In [ ]:
df = pd.DataFrame(data_label, columns = data_label[0].keys(), index = None)

In [ ]:
df["label"] = df["label"].map({"HOF":1,"NONE":0})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2092 entries, 0 to 2091
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   post     2092 non-null   object
 1   context  2092 non-null   object
 2   text     2092 non-null   object
 3   label    2092 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 65.5+ KB


In [ ]:
df.to_pickle("test_hierarchial.pkl")

In [ ]:
test = pd.read_pickle("test_hierarchial.pkl")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

epochs = 4 

print("Testing....")
for epoch in range(epochs):
    predictions = []
    labels = []
    print("==========Epochs:{}===========".format(epoch))  

    for post, context,text, label in zip(test["post"], test["context"], test["text"], test["label"]):

      logits = model.forward(post, context,text, label)

      y_pred = torch.argmax(logits, dim = -1).item()

      predictions.append(y_pred)
      labels.append(label)
    print("Accuracy: {}".format(accuracy_score(predictions, labels)))

Epochs:0


KeyboardInterrupt: ignored